# Interpolacja 

### Imports

In [2]:
import math
import numpy as np
import sympy as sp
from tabulate import tabulate

### Zadanie 1
Stablicuj następujące funkcje: sqrt(x), sin(x), x^3+2x w czterech punktach należących do przedziału 0 do 10.

In [21]:
def into_table(fun, fun_name, min_val, max_val, num):
    x = []
    y = []
    tab = []
    section = (max_val - min_val) / (num - 1.0)
    
    for i in np.arange(min_val, max_val + 0.1, section):
        x.append(i)
        y.append(fun(i))
        tab.append([i, fun(i)])
        
    return x, y, tabulate(tab, headers=["x", fun_name],
                          tablefmt="fancy_grid", floatfmt=".10f")


print(into_table(math.sqrt, "sqrt(x)", 0, 10, 4)[2])
print(into_table(math.sin, "sin(x)", 0, 10, 4)[2])
print(into_table(lambda x: x**3 + 2*x, "x^3 + 2x", 0, 10, 4)[2])

╒═══════════════╤══════════════╕
│             x │      sqrt(x) │
╞═══════════════╪══════════════╡
│  0.0000000000 │ 0.0000000000 │
├───────────────┼──────────────┤
│  3.3333333333 │ 1.8257418584 │
├───────────────┼──────────────┤
│  6.6666666667 │ 2.5819888975 │
├───────────────┼──────────────┤
│ 10.0000000000 │ 3.1622776602 │
╘═══════════════╧══════════════╛
╒═══════════════╤═══════════════╕
│             x │        sin(x) │
╞═══════════════╪═══════════════╡
│  0.0000000000 │  0.0000000000 │
├───────────────┼───────────────┤
│  3.3333333333 │ -0.1905679629 │
├───────────────┼───────────────┤
│  6.6666666667 │  0.3741512306 │
├───────────────┼───────────────┤
│ 10.0000000000 │ -0.5440211109 │
╘═══════════════╧═══════════════╛
╒═══════════════╤═════════════════╕
│             x │          x^3+2x │
╞═══════════════╪═════════════════╡
│  0.0000000000 │    0.0000000000 │
├───────────────┼─────────────────┤
│  3.3333333333 │   43.7037037037 │
├───────────────┼─────────────────┤
│  6.666666

### Zadanie 2 
Napisz algorytm znajdujący wielomian interpolujący Lagrange dla powyższych stablicowanych funkcji.

In [22]:
def interpolate(x_arr, y_arr, xi):
    if len(x_arr) != len(y_arr):
        return -1
    
    res = 0.0
    for i in range(len(x_arr)):
        temp = 1
        for j in range(len(x_arr)):
            if i != j:
                temp = temp * ((xi - x_arr[j]) / (x_arr[i] - x_arr[j]))
        res += temp * y_arr[i]
    return sp.simplify(res)


x, y, _ = into_table(math.sqrt, "sqrt(x)", 0, 10, 4)
print("Lagrange polynomial of sqrt(x):", interpolate(x, y, sp.symbols('x')))

x, y, _ = into_table(math.sin, "sin(x)", 0, 10, 4)
print("Lagrange polynomial of sin(x):", interpolate(x, y, sp.symbols('x')))

x, y, _ = into_table(lambda x: x**3 + 2*x, "x^3 + 2x", 0, 10, 4)
print("Lagrange polynomial of x^3 + 2x:", interpolate(x, y, sp.symbols('x')))

('Lagrange polynomial of sqrt(x):', x*(0.00402091444262344*x**2 - 0.0883364112915617*x + 0.797500434670112))
('Lagrange polynomial of sin(x):', x*(-0.0100718041105327*x**2 + 0.134705963139825*x - 0.394281331433923))
('Lagrange polynomial of x^3 + 2x:', x*(1.0*x**2 - 7.105427357601e-15*x + 2.0))


### Zadanie 3
Porównaj wartość dokładną z wynikiem interpolacji dla punktów znajdujących się pomiędzy węzłami wielomianu (w połowie odległości) interpolującego. Oszacuj dokładność interpolacji. 


In [37]:
x_sqrt, y_sqrt, sqrt_t = into_table(math.sqrt, "sqrt(x)", 0, 10, 4)
x_sin, y_sin, sin_t = into_table(math.sin, "sin(x)", 0, 10, 4)
x_pol, y_pol, p_t = into_table(lambda x: x**3 + 2*x, "x^3 + 2x", 0, 10, 4)

l_sqrt = lambda x: interpolate(x_sqrt, y_sqrt, x)
l_sin = lambda x: interpolate(x_sin, y_sin, x)
l_pol = lambda x: interpolate(x_pol, y_pol, x)


def compare(fun1, fun2, fname, min, max, n):
    tab = []
    section = (max - min) / float(n)

    
    min_val = section / 2 + min
    max_val = section / 2 + max
    
    for i in np.arange(min_val, max_val, section):
        tab.append([i, fun1(i), fun2(i), abs(fun2(i) - fun1(i)) / abs(fun1(i))])
        
    print(tabulate(tab, headers=['x', fname, 'Lagrange interpolation' + fname,
                                 'Relative error'], tablefmt="fancy_grid", floatfmt=".10f"))


compare(math.sqrt, l_sqrt , "sqrt(x)", 0, 10, 4)
compare(math.sin, l_sin , "sin(x)", 0, 10, 4)
compare(lambda x: x**3 + 2*x, l_pol , "x^3+2x", 0, 10, 4)

╒══════════════╤══════════════╤═════════════════════════════════╤══════════════════╕
│            x │      sqrt(x) │   Lagrange interpolationsqrt(x) │   Relative error │
╞══════════════╪══════════════╪═════════════════════════════════╪══════════════════╡
│ 1.2500000000 │ 1.1180339887 │                    0.8667032492 │     0.2247970474 │
├──────────────┼──────────────┼─────────────────────────────────┼──────────────────┤
│ 3.7500000000 │ 1.9364916731 │                    1.9604362563 │     0.0123649296 │
├──────────────┼──────────────┼─────────────────────────────────┼──────────────────┤
│ 6.2500000000 │ 2.5000000000 │                    2.5154052157 │     0.0061620863 │
├──────────────┼──────────────┼─────────────────────────────────┼──────────────────┤
│ 8.7500000000 │ 2.9580398915 │                    2.9085708565 │     0.0167235862 │
╘══════════════╧══════════════╧═════════════════════════════════╧══════════════════╛
╒══════════════╤═══════════════╤════════════════════════════════╤

### Zadanie 4
Powtórz powyższe kroki dla 3, 5 i 8 węzłów interpolacji - podsumuj badania. 

In [38]:
def compare2(min, max, n):
    x_sqrt, y_sqrt, sqrt_t = into_table(math.sqrt, "sqrt(x)", min, max, n)
    x_sin, y_sin, sin_t = into_table(math.sin, "sin(x)", min, max, n)
    x_pol, y_pol, p_t = into_table(lambda x: x**3 + 2*x, "x^3 + 2x", min, max, n)

    l_sqrt = lambda x: interpolate(x_sqrt, y_sqrt, x)
    l_sin = lambda x: interpolate(x_sin, y_sin, x)
    l_pol = lambda x: interpolate(x_pol, y_pol, x)

    compare(math.sqrt, l_sqrt , "sqrt(x)", min, max, n)
    compare(math.sin, l_sin , "sin(x)", min, max, n)
    compare(lambda x: x**3 + 2*x, l_pol , "x^3+2x", min, max, n)
    
print("3 nodes:")
compare2(0, 10, 3)
print("5 nodes:")
compare2(0, 10, 5)
print("8 nodes:")
compare2(0, 10, 8)

3 nodes:
╒══════════════╤══════════════╤═════════════════════════════════╤══════════════════╕
│            x │      sqrt(x) │   Lagrange interpolationsqrt(x) │   Relative error │
╞══════════════╪══════════════╪═════════════════════════════════╪══════════════════╡
│ 1.6666666667 │ 1.2909944487 │                    0.8908958030 │     0.3099150783 │
├──────────────┼──────────────┼─────────────────────────────────┼──────────────────┤
│ 5.0000000000 │ 2.2360679775 │                    2.2360679775 │     0.0000000000 │
├──────────────┼──────────────┼─────────────────────────────────┼──────────────────┤
│ 8.3333333333 │ 2.8867513459 │                    2.9990809098 │     0.0389121024 │
╘══════════════╧══════════════╧═════════════════════════════════╧══════════════════╛
╒══════════════╤═══════════════╤════════════════════════════════╤══════════════════╕
│            x │        sin(x) │   Lagrange interpolationsin(x) │   Relative error │
╞══════════════╪═══════════════╪════════════════════════